In [1]:
%env THEANO_FLAGS=device=gpu0,floatX=float32
import theano
import theano.tensor as T
import lasagne

env: THEANO_FLAGS=device=gpu0,floatX=float32


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5110)


In [2]:
lasagne.layers.set_all_param_values

<function lasagne.layers.helper.set_all_param_values>

In [3]:
MUSIC_SHAPE = (None, 20, 200)
START_POSITION_SHAPE = (None, 38, 3)
SHIFT_SHAPE = (None, 20, 38*3)

In [4]:
input_music_var = T.tensor3("Music input")
input_shift_var = T.tensor3("Shift input")

input_position_var = T.tensor3("Start position")
delta_mov_var = T.matrix("Delta moving")

In [9]:
music_input = lasagne.layers.InputLayer(shape=MUSIC_SHAPE, input_var=input_music_var)
music_input = lasagne.layers.BatchNormLayer(music_input)
music_dims = lasagne.layers.DimshuffleLayer(music_input, (0, 2, 1))
music_conv = lasagne.layers.Conv1DLayer(music_dims, 256, 10, name="Music")
music_conv = lasagne.layers.batch_norm(music_conv)
# music_pool = lasagne.layers.GlobalPoolLayer(music_conv, T.max)
print("Music conv: ", music_conv.output_shape)

shift_input = lasagne.layers.InputLayer(shape=SHIFT_SHAPE, input_var=input_shift_var)
shift_input = lasagne.layers.BatchNormLayer(shift_input)
shift_dims = lasagne.layers.DimshuffleLayer(shift_input, (0, 2, 1))
shift_conv = lasagne.layers.Conv1DLayer(shift_dims, 256, 10, name="shift_conv")
shift_conv = lasagne.layers.batch_norm(shift_conv)
# shift_pool = lasagne.layers.GlobalPoolLayer(music_conv, T.max)
print("Shift conv: ", shift_conv.output_shape)

position_input = lasagne.layers.InputLayer(shape=START_POSITION_SHAPE, input_var=input_position_var)
position_input = lasagne.layers.BatchNormLayer(position_input)
position_dims = lasagne.layers.DimshuffleLayer(position_input, (0, 2, 1))
position_conv = lasagne.layers.Conv1DLayer(position_dims, 128, 2, name="Conv0")
position_conv = lasagne.layers.batch_norm(position_conv)
position_pool = lasagne.layers.Pool1DLayer(position_conv, 2, name="Pool0")
position_conv = lasagne.layers.Conv1DLayer(position_pool, 512, 2, name="Conv1")
position_conv = lasagne.layers.batch_norm(position_conv)
position_pool = lasagne.layers.GlobalPoolLayer(position_conv, pool_function=T.max, name="Gpool")
print("Position pool: ", position_pool.output_shape)

lstm_music = lasagne.layers.LSTMLayer(music_conv, 512, only_return_final=True, name="Music LSTM")
lstm_music = lasagne.layers.batch_norm(lstm_music)
# lstm_music = lasagne.layers.ReshapeLayer(lstm_music, (-1, 1, 512))

lstm_shift = lasagne.layers.LSTMLayer(shift_conv, 512, hid_init=position_pool, only_return_final=True, name="Shift LSTM")
lstm_shift = lasagne.layers.batch_norm(lstm_shift)
# lstm_shift = lasagne.layers.ReshapeLayer(lstm_shift, (-1, 1, 512))

conc = lasagne.layers.ConcatLayer([lstm_music, lstm_shift], axis=1)
print("Conc: ", conc.output_shape)
dense_0 = lasagne.layers.DenseLayer(conc, 256, nonlinearity=lasagne.nonlinearities.sigmoid, name="Dense1")
output = lasagne.layers.DenseLayer(dense_0, 38*3, nonlinearity=lasagne.nonlinearities.linear, name="Conv2")
print("Output: ", output.output_shape)

reshape_output = lasagne.layers.ReshapeLayer(output, (-1, 38, 3))

Music conv:  (None, 256, 11)
Shift conv:  (None, 256, 11)
Position pool:  (None, 512)
Conc:  (None, 1024)
Output:  (None, 114)


In [5]:
def gr(l):
    theano.grad(lasagne.layers.get_output(l).sum(), lasagne.layers.get_all_params(l))

In [10]:
weights = lasagne.layers.get_all_params(output, trainable=True)
print(weights)

[beta, gamma, Music.W, Music_bn.beta, Music_bn.gamma, Music LSTM.W_in_to_ingate, Music LSTM.W_hid_to_ingate, Music LSTM.b_ingate, Music LSTM.W_in_to_forgetgate, Music LSTM.W_hid_to_forgetgate, Music LSTM.b_forgetgate, Music LSTM.W_in_to_cell, Music LSTM.W_hid_to_cell, Music LSTM.b_cell, Music LSTM.W_in_to_outgate, Music LSTM.W_hid_to_outgate, Music LSTM.b_outgate, Music LSTM.W_cell_to_ingate, Music LSTM.W_cell_to_forgetgate, Music LSTM.W_cell_to_outgate, Music LSTM_bn.beta, Music LSTM_bn.gamma, beta, gamma, shift_conv.W, shift_conv_bn.beta, shift_conv_bn.gamma, beta, gamma, Conv0.W, Conv0_bn.beta, Conv0_bn.gamma, Conv1.W, Conv1_bn.beta, Conv1_bn.gamma, Shift LSTM.W_in_to_ingate, Shift LSTM.W_hid_to_ingate, Shift LSTM.b_ingate, Shift LSTM.W_in_to_forgetgate, Shift LSTM.W_hid_to_forgetgate, Shift LSTM.b_forgetgate, Shift LSTM.W_in_to_cell, Shift LSTM.W_hid_to_cell, Shift LSTM.b_cell, Shift LSTM.W_in_to_outgate, Shift LSTM.W_hid_to_outgate, Shift LSTM.b_outgate, Shift LSTM.W_cell_to_ingat

In [7]:
gr(output)

In [7]:
out = lasagne.layers.get_output(output)
# fun = theano.function([input_music_var, input_shift_var, input_position_var], out, allow_input_downcast=True)

In [22]:
bio_check = theano.shared(0., name="bio_check")
loss = T.sum([bio_check, lasagne.objectives.squared_error(out, delta_mov_var).sum()])
# print(loss)

In [14]:
update = lasagne.updates.adam(loss, weights, learning_rate=0.0001)

In [15]:
train_ = theano.function([input_music_var, input_shift_var, input_position_var, delta_mov_var], updates=update, allow_input_downcast=True)

In [12]:
import numpy as np

In [13]:
mus = np.random.random((100, 200, 200))
st_pos = np.random.random((100, 38, 3))
shift_ = np.random.random((100, 200, 38*3))
delta_mov = np.random.random((100, 38*3))

In [16]:
train_(mus, shift_, st_pos, delta_mov)

[]